In [7]:
import os

In [8]:
! pip list

Package                       Version         Editable project location
----------------------------- --------------- -----------------------------------------------------------
absl-py                       2.0.0
aiobotocore                   2.5.0
aiofiles                      22.1.0
aiohttp                       3.8.5
aiohttp-retry                 2.8.3
aioitertools                  0.7.1
aiosignal                     1.2.0
aiosqlite                     0.18.0
alabaster                     0.7.12
alembic                       1.13.0
altair                        5.0.1
amqp                          5.2.0
anaconda-anon-usage           0.4.2
anaconda-catalogs             0.2.0
anaconda-client               1.12.1
anaconda-cloud-auth           0.1.3
anaconda-navigator            2.5.0
anaconda-project              0.11.1
antlr4-python3-runtime        4.9.3
anyio                         3.5.0
appdirs                       1.4.4
applaunchservices             0.3.0
appnope                 

In [12]:
%pwd

'/Users/ayushyajnik/Desktop/Python/lapse_rate_app/research'

In [13]:
os.chdir("../")

In [14]:
%pwd

'/Users/ayushyajnik/Desktop/Python/lapse_rate_app'

In [15]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [17]:
from src.No_More_Lapses.constants import *
from src.No_More_Lapses.utils.common import read_yaml, create_directories

In [18]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [ ]:
import os
import zipfile
import gdown
from src.No_More_Lapses import logger
from src.No_More_Lapses.utils.common import get_size

In [47]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self) -> str:
        '''
        Fetch data from the url
        '''
        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            # Extract the file ID from the Google Drive URL
            if "drive.google.com" in dataset_url:
                if "/file/d/" in dataset_url:
                    # Format: https://drive.google.com/file/d/{FILE_ID}/view
                    file_id = dataset_url.split("/file/d/")[1].split("/")[0]
                elif "id=" in dataset_url:
                    # Format: https://drive.google.com/uc?id={FILE_ID}
                    file_id = dataset_url.split("id=")[1].split("&")[0]
                else:
                    # Try to extract from your current approach as fallback
                    file_id = dataset_url.split("/")[-2]
                
                logger.info(f"Extracted file_id: {file_id}")
                
                # Use gdown's direct ID download method
                gdown.download(id=file_id, output=str(zip_download_dir), quiet=False)
            else:
                # For non-Google Drive URLs, you might want to use a different method
                import requests
                response = requests.get(dataset_url)
                with open(zip_download_dir, 'wb') as f:
                    f.write(response.content)
            
            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")
            return zip_download_dir
        except Exception as e:
            logger.error(f"Error downloading file: {str(e)}")
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [48]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    #data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-03-22 19:30:48,394: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-03-22 19:30:48,396: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-22 19:30:48,397: INFO: common: created directory at: artifacts]
[2025-03-22 19:30:48,398: INFO: common: created directory at: artifacts/data_ingestion]
[2025-03-22 19:30:48,399: INFO: 1508000803: Downloading data from https://docs.google.com/spreadsheets/d/1mW6Agx4ueXhuM6D9yrUlt2uc_C1nHN5o/edit?usp=sharing into file artifacts/data_ingestion/lapse-rate.xlsx]
[2025-03-22 19:30:49,002: INFO: 1508000803: Downloaded data from https://docs.google.com/spreadsheets/d/1mW6Agx4ueXhuM6D9yrUlt2uc_C1nHN5o/edit?usp=sharing into file artifacts/data_ingestion/lapse-rate.xlsx]
